### ridgeClassifier概述

1. 是岭回归器的分类器变体，带线性核的最小二乘支持向量机。使用了最小二乘作为损失函数，而不是传统的逻辑或铰链损失(最大边界损失)
2. ridgeClassifier只需要一次计算投影矩阵，比logisticRegression快很多
3. 


In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

In [3]:
train_df=pd.read_csv("/data/aig/public/wuchunsheng/NLP/competition/data/cnews/train_set.csv",sep="\t",nrows=15000)


In [55]:
vectorizer=CountVectorizer(max_features=2000,ngram_range=(1,3))
train_test=vectorizer.fit_transform(train_df["text"])

In [56]:
clf=RidgeClassifier()

In [57]:
clf.fit(train_test[:10000],train_df["label"].values[:10000])

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [58]:
val_pred=clf.predict(train_test[10000:])
print(f1_score(train_df["label"].values[10000:],val_pred,average="macro"))

0.779904503620177


In [59]:
## https://blog.csdn.net/weixin_38278334/article/details/82320307
## https://blog.csdn.net/LOLUN9/article/details/106012418/
## https://blog.csdn.net/LOLUN9/article/details/106012418/

## gridsearchCV 的参数使用: https://www.biaodianfu.com/gridsearchcv.html

### 网格搜索

In [60]:
from sklearn.model_selection import GridSearchCV
## alpha, 正数_浮点数，正则化强度
param_grid=[
    {"alpha":[1.0,0.1,0.01],
    "normalize":[False,True],
    "max_iter":[1,5,10],
    "class_weight":[None,"balanced"]}
]

In [61]:
grid_search=GridSearchCV(clf,param_grid,cv=5)

In [62]:
grid_search.fit(train_test[:10000],train_df["label"].values[:10000])

GridSearchCV(cv=5, error_score=nan,
             estimator=RidgeClassifier(alpha=1.0, class_weight=None,
                                       copy_X=True, fit_intercept=True,
                                       max_iter=None, normalize=False,
                                       random_state=None, solver='auto',
                                       tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid=[{'alpha': [1.0, 0.1, 0.01],
                          'class_weight': [None, 'balanced'],
                          'max_iter': [1, 5, 10], 'normalize': [False, True]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [63]:
### 通过测试max_iter貌似是越大越好？
grid_search.best_params_

{'alpha': 0.01, 'class_weight': None, 'max_iter': 10, 'normalize': True}

### 使用grid_search的结果

In [68]:
clf=RidgeClassifier(alpha=0.01,max_iter=10,normalize=True)

In [69]:
clf.fit(train_test[:10000],train_df["label"].values[:10000])

RidgeClassifier(alpha=0.01, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=10, normalize=True, random_state=None, solver='auto',
                tol=0.001)

In [70]:
val_pred=clf.predict(train_test[10000:])
print(f1_score(train_df["label"].values[10000:],val_pred,average="macro"))

0.7886200060176475


In [67]:
help(RidgeClassifier)

Help on class RidgeClassifier in module sklearn.linear_model._ridge:

class RidgeClassifier(sklearn.linear_model._base.LinearClassifierMixin, _BaseRidge)
 |  RidgeClassifier(alpha=1.0, fit_intercept=True, normalize=False, copy_X=True, max_iter=None, tol=0.001, class_weight=None, solver='auto', random_state=None)
 |  
 |  Classifier using Ridge regression.
 |  
 |  This classifier first converts the target values into ``{-1, 1}`` and
 |  then treats the problem as a regression task (multi-output regression in
 |  the multiclass case).
 |  
 |  Read more in the :ref:`User Guide <ridge_regression>`.
 |  
 |  Parameters
 |  ----------
 |  alpha : float, default=1.0
 |      Regularization strength; must be a positive float. Regularization
 |      improves the conditioning of the problem and reduces the variance of
 |      the estimates. Larger values specify stronger regularization.
 |      Alpha corresponds to ``C^-1`` in other linear models such as
 |      LogisticRegression or LinearSVC.